In [4]:
!pip install -q duckdb
!pip install -q deltalake

StatementMeta(, fc5fa820-7eb6-4348-b83a-91e4cc9c042f, 8, Finished, Available)

In [5]:
#the function read all the tables in the default Lakehouse and save it in a Table Delta_stats, change the url to reflect your destination
import duckdb 
import pandas as pd
from deltalake import DeltaTable
import os
from datetime import datetime
from deltalake.writer import write_deltalake
def get_stats(Source,Destination):
    appended_data = []
    List_tables = [ f.name for f in os.scandir(Source) if f.is_dir() ]
    stat_table = os.path.basename(os.path.normpath(Destination))
    if stat_table in List_tables: List_tables.remove(stat_table)
    for tbl in List_tables :
        dt = DeltaTable(Source+ tbl)
        xx= dt.get_add_actions(flatten=True).to_pandas()
        if 'tags.VORDER' in xx.columns:
             vorder =True
        else :
             vorder =False
        deltalog=pd.DataFrame([[tbl,xx['size_bytes'].sum(),vorder]],columns=['tbl','size_bytes','vorder'])
        delta = [Source+tbl + "/" + i for i in dt.files()]
        duckdb.sql(f'from parquet_scan({delta})').to_view(tbl)
        df= duckdb.sql(f'''
        with x as (
        select file_name,num_rows,num_row_groups, '{tbl}' as tbl from parquet_file_metadata({delta})
        )
        select x.tbl, file_name,num_rows,num_row_groups, ceil(size_bytes/(1024*1024)) as size,vorder from x
        left join deltalog as y
        on x.tbl=y.tbl
         ''').df()
        appended_data.append(df)
    appended_data = pd.concat(appended_data,ignore_index=True)
    appended_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    write_deltalake(Destination,appended_data,mode="overwrite",storage_options={"allow_unsafe_rename":"true"})

    dt = DeltaTable(Destination)
    delta = [Destination + "/" + i for i in dt.files()]
    duckdb.sql(f'from parquet_scan({delta})').to_view("xxx")

    duckdb.sql('''
    select tbl,
    sum(num_rows) as total_rows,
    count(*) as num_files,
    sum(num_row_groups) as num_row_group,
    cast(ceil(total_rows/num_row_group) as integer) as Average_row_group,
    min(size) as file_size_MB,
    vorder
    from xxx 
    group by all
    order by total_rows desc
    ''').show()

StatementMeta(, fc5fa820-7eb6-4348-b83a-91e4cc9c042f, 9, Finished, Available)

In [6]:
get_stats('/lakehouse/default/Tables/','/lakehouse/default/Files/xx_deltastats')

StatementMeta(, fc5fa820-7eb6-4348-b83a-91e4cc9c042f, 10, Finished, Available)

┌──────────────┬────────────┬───────────┬───────────────┬───────────────────┬──────────────┬─────────┐
│     tbl      │ total_rows │ num_files │ num_row_group │ Average_row_group │ file_size_MB │ vorder  │
│   varchar    │   int128   │   int64   │    int128     │       int32       │    double    │ boolean │
├──────────────┼────────────┼───────────┼───────────────┼───────────────────┼──────────────┼─────────┤
│ scada_spark  │  230363649 │         7 │            35 │           6581819 │       1565.0 │ true    │
│ scada_python │  230363649 │        30 │           235 │            980271 │       2051.0 │ false   │
│ calendar     │       2467 │         1 │             1 │              2467 │          1.0 │ true    │
│ duid         │        606 │         1 │             1 │               606 │          1.0 │ true    │
└──────────────┴────────────┴───────────┴───────────────┴───────────────────┴──────────────┴─────────┘

